# Sudoku solver using numpy

In [ ]:
import numpy as np

In [ ]:
# constants
BOARD_SIZE = 9
SECTOR_SIZE = 3
VALID_VALUES = '123456789'

---

## Helper functions

### Numpy vectorized functions

In [ ]:
"""Returns the length of the contents of each cell."""
get_length = np.vectorize(len)

"""Looks for the specified string pattern in each cell."""
cell_contains = np.vectorize(lambda x, y : y in x)

"""Remove a specific string from every location in a numpy array."""
strip_string = np.vectorize(lambda x, y : x.replace(y,''))

"""Remove every character of a pattern from every location in a
numpy array unless that cell exactly matches the pattern."""
strip_pattern = np.vectorize(lambda x, y : x if x == y else ''.join([c for c in x if c not in y]))

### Generic helpers

In [ ]:
def clean_neighbors(arr, row, col, value):
    """Removes the specified value from the row, column, and sector of the
    specified coordinates."""

    # remove the value from every cell in the row
    arr[row,:] = strip_string(arr[row,:],value)

    # remove the value from every cell in the column
    arr[:,col] = strip_string(arr[:,col],value)

    # remove the value from every cell in the sector
    r_start = row // SECTOR_SIZE * SECTOR_SIZE
    c_start = col // SECTOR_SIZE * SECTOR_SIZE
    r_end = r_start + SECTOR_SIZE
    c_end = c_start + SECTOR_SIZE
    arr[r_start:r_end,c_start:c_end] = strip_string(
        arr[r_start:r_end,c_start:c_end],value
    )


def get_sector_by_number(arr, sector):
    """Returns one sector of an array by index between zero and BOARD_SIZE."""
    row = sector // SECTOR_SIZE * SECTOR_SIZE
    col = SECTOR_SIZE * (sector % SECTOR_SIZE)
    return arr[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE]


def convert_sector_row(sector, row):
    """Returns the absolute row number when given a row number within a sector."""
    return sector // SECTOR_SIZE * SECTOR_SIZE + row


def convert_sector_col(sector, col):
    """Returns the absolute column number when given a column number with a sector."""
    return (sector % SECTOR_SIZE) * SECTOR_SIZE + col


def print_one_board(arr):
    """Print a single Sudoku board with sector borders."""
    board = arr.flatten()
    width = np.max(get_length(board)) + 2
    line = '+' + '-'*(width*3) + '+' + '-'*(width*3) + '+' + '-'*(width*3) + '+'
    for idx, cell in enumerate(board):
        if not idx % (SECTOR_SIZE**3):
            print(line)
        if not idx % SECTOR_SIZE:
            print('|', end='')
        print(F'{cell:^{width}s}', end='')
        if not ((idx + 1) % BOARD_SIZE):
            print('|')
    print(line)

def print_two_boards(arr1, arr2):
    """Prints two Sudoku boards side by side with sector borders."""
    width1 = np.max(get_length(arr1)) + 1
    width1 = 3 if width1 < 3 else width1
    line1 = '+' + '-'*(width1*3) + '+' + '-'*(width1*3) + '+' + '-'*(width1*3) + '+'
    width2 = np.max(get_length(arr2)) + 1
    width2 = 3 if width1 < 3 else width2
    line2 = '+' + '-'*(width2*3) + '+' + '-'*(width2*3) + '+' + '-'*(width2*3) + '+'

    for row in range(BOARD_SIZE):
        if not row % SECTOR_SIZE:
            print(line1 + '  ' + line2)
        
        for col in range(BOARD_SIZE):
            if not col % SECTOR_SIZE:
                print('|', end='')
            print(F'{arr1[row,col]:^{width1}s}', end='')

        print('|  ', end='')

        for col in range(BOARD_SIZE):
            if not col % SECTOR_SIZE:
                print('|', end='')
            print(F'{arr2[row,col]:^{width2}s}', end='')

        print('|')
    print(line1 + '  ' + line2)
                
    

In [ ]:
def board_valid(arr, verbose=False):
    """Returns the number of errors in the board or zero if the board is solved."""
    counter = 0
    errors = []
    for pattern in '123456789':
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(arr[idx,:],pattern)) != 1:
                count = np.count_nonzero(cell_contains(arr[idx,:],pattern))
                errors.append(F"There are {count} instances of '{pattern}' in zero-based row {idx}.")
                counter += 1
            if np.count_nonzero(cell_contains(arr[:,idx],pattern)) != 1:
                count = np.count_nonzero(cell_contains(arr[:,idx],pattern))
                errors.append(F"There are {count} instances of '{pattern}' in zero-based column {idx}.")
                counter += 1
            if np.count_nonzero(cell_contains(get_sector_by_number(arr,idx),pattern)) != 1:
                count = np.count_nonzero(cell_contains(get_sector_by_number(arr,idx),pattern))
                errors.append(F"There are {count} instances of '{pattern}' in zero-based sector {idx}.")
                counter += 1
    if verbose and counter:
        print('\n'.join(errors))
    return counter

### Algorithms

In [ ]:
def handle_singletons(solved,wip):
    """Transfers all singletons from the WIP array to the final array, cleans
    that singleton pattern from all neighbors, and returns the number of singletons
    found. This will run until there are no singletons left in the WIP array.
    
    Keyword arguments:
    solved -- numpy array containing only solved elements
    wip -- work in progress numpy array containing possible values for each cell
    """
    change_counter = 0
    coordinate_list = np.argwhere(get_length(wip)==1)
    while coordinate_list.size:
        for idc in coordinate_list:
            row, col = idc
            singleton = wip[row,col]
            # update the solved array with the singleton value
            solved[row,col] = singleton

            clean_neighbors(wip,row,col,singleton)
            change_counter += 1

        # check for more singletons
        coordinate_list = np.argwhere(get_length(wip)==1)

    # return the number of times the while loop executed
    return change_counter

In [ ]:
def handle_disguised_singletons(solved, wip, discovered=0):
    """Sets any cells containing disguised singleton values to the actual singleton
    value and returns the number of singletons found. Runs recursively until no
    additional disguised singletons are found."""
    altered_coords = set({})
    for val in '123456789':
        # search every row
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(wip[idx,:],val)) == 1:
                col = np.argwhere(cell_contains(wip[idx,:],val))[0][0]
                wip[idx,col] = ''
                solved[idx,col] = val
                altered_coords.add((idx,col))
                clean_neighbors(wip,idx,col,val)

        # search every column
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(wip[:,idx],val)) == 1:
                row = np.argwhere(cell_contains(wip[:,idx],val))[0][0]
                wip[row,idx] = ''
                solved[row,idx] = val
                altered_coords.add((row,idx))
                clean_neighbors(wip,row,idx,val)

        # search every sector
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(get_sector_by_number(wip,idx),val)) == 1:
                coords = np.argwhere(cell_contains(get_sector_by_number(wip,idx),val))
                row = coords[0][0] + SECTOR_SIZE * (idx // SECTOR_SIZE)
                col = coords[0][1] + SECTOR_SIZE * (idx % SECTOR_SIZE)
                wip[row,col] = ''
                solved[row,col] = val
                altered_coords.add((row,col))
                clean_neighbors(wip,row,col,val)

    if len(altered_coords) != discovered:
        return len(altered_coords) + handle_disguised_singletons(solved,wip,len(altered_coords))
    else:
        return len(altered_coords)

In [ ]:
def handle_rowcol_owners(arr):
    """Finds sector values that exist only in one row or only in one column and removes
    that value from that row or column in other sectors. Returns the number of values
    changed. Runs recursively."""
    # take a snapshot for later comparison to count changes
    snapshot = np.copy(arr)
    for idx in range(BOARD_SIZE):
        for value in VALID_VALUES:
            # get coordinates for all cells containing this value
            coordinate_list = np.argwhere(cell_contains(get_sector_by_number(arr,idx), value))
            # see if all these coordinates are in one row or one column
            rows = set({})
            cols = set({})
            for coord in coordinate_list:
                rows.add(coord[0])
                cols.add(coord[1])

            # if len(rows) is 1 then every {value} in the current sector is in one row
            if len(rows) == 1:
                # print(F"sector {idx} row {coord[0]}: {value}")
                actual_row = convert_sector_row(idx, coord[0])
                actual_col_list = [convert_sector_col(idx, x) for x in cols]
                numpy_iterator = np.nditer(arr[actual_row,:], flags=['multi_index'], op_flags=['readwrite'])
                for cell in numpy_iterator:
                    if numpy_iterator.multi_index[0] not in actual_col_list:
                        cell[...] = str(cell).replace(value,'')
            
            # if len(cols) is 1 then every {value} in the current sector is in one column
            if len(cols) == 1:
                # print(F"sector {idx} col {coord[1]}: {value}")
                actual_col = convert_sector_col(idx, coord[1])
                actual_row_list = [convert_sector_row(idx, x) for x in rows]
                numpy_iterator = np.nditer(arr[:,actual_col], flags=['multi_index'], op_flags=['readwrite'])
                for cell in numpy_iterator:
                    if numpy_iterator.multi_index[0] not in actual_row_list:
                        cell[...] = str(cell).replace(value,'')

    # compare the [maybe] updated array with the snapshot to detect changes
    altered_cells = np.sum(arr != snapshot)
    if altered_cells:
        return altered_cells + handle_rowcol_owners(arr)
    else:
        return 0

In [ ]:
def handle_naked_groups(arr):
    """Identifies naked groups (twins, triples, quads, etc.) in a row, column, or sector
    and removes those group values from other cells in that row, column, or sector."""

    def get_special_patterns(arr):
        """Returns valid patterns for existing naked groups. A pattern of length N
        must exist exactly N times in the array to be considered a naked group."""
        return_list = []
        pattern_dict = dict()
        numpy_iterator = np.nditer(arr, flags=['multi_index'], op_flags=['readwrite'])
        for cell in numpy_iterator:
            pattern_dict[str(cell)] = pattern_dict.get(str(cell),0) + 1
        for k,v in pattern_dict.items():
            if len(k) == v:
                return_list.append(k)
        return return_list

    # take a snapshot for later comparison to count changes
    snapshot = np.copy(arr)
    for row in range(BOARD_SIZE):
        for pattern in get_special_patterns(arr[row,:]):
            strip_pattern(arr[row,:],pattern)
    for col in range(BOARD_SIZE):
        for pattern in get_special_patterns(arr[:,col]):
            strip_pattern(arr[:,col],pattern)
    for row in range(0,BOARD_SIZE,3):
        for col in range(0,BOARD_SIZE,3):
            for pattern in get_special_patterns(arr[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE]):
                strip_pattern(arr[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE],pattern)

    altered_cells = np.sum(arr != snapshot)
    if altered_cells:
        return altered_cells + handle_naked_groups(arr)
    else:
        return 0

## Solve multiple puzzles

In [ ]:
puzzle_dict = dict(
    easy1 = '5..98.67.6......31.2.613.4..968.21.7..8..5.9.7.319....962.7..1.1.5...76..7.5..9..',
    medium1 = '.29.71..3..8...6..3...5....5.....97......4...4.75.8..1.6.42.3..2..9....6.916...52',
    hard1 = '..791.5....1.....3..9.4...2.4...83.....3.1....6..5...8.2..9...5...........4.8..7.',
    hard2 = '.6......4.5..61.8..1..9...32...8...7...6.4...9..7...4..9..7.5..3...1...8.........',
    hard3 = '...34..1..5...6...47....56.6.........341...8....5.8..756...3......68.1..1.7254...',
    hard4 = '.7.8..3..9...7.5......69...8..6.4..2.9......6.1.5....4.8.4..1.............27....5',
    hard5 = '82.5.........3.257...67.9..4.61...3...........5..8419.9.2.1......57...2.......561',
    nrichmathsorg = '....3...6...6.2..58..4.372..49.........9.4...1.3.6.9.45.4..6.8.....4.1.771.....4.',
    # expert1 = '.....2..3.4...16.7..1.....4.8.1.....43.....62...7..............6.98.73.....3.47..'
)

for name, puzzle in puzzle_dict.items():
    # initialize an empty board and a full "work in progress" array
    possible = np.full((9,9),'123456789')
    board = np.full((9,9),'')

    # load the puzzle into the array of possibilities
    for idx, value in enumerate(puzzle):
        if value in VALID_VALUES:
            row = idx // BOARD_SIZE
            col = idx - row * BOARD_SIZE
            possible[row,col] = value

    while changes := handle_singletons(board,possible):
        changes += handle_rowcol_owners(possible)
        changes += handle_naked_groups(possible)
        changes += handle_disguised_singletons(board,possible)

    errors = board_valid(board)
    if errors:
        print(F"Puzzle {name} has {errors} errors.")
    else:
        print(F"Puzzle {name} solved!")


In [ ]:
# def hidden_naked_groups(arr):

#     def grow_pattern(arr):
#         """Returns a new array where each element is one character longer."""
#         expanded = []
#         for pattern in arr:
#             next_index = VALID_VALUES.find(pattern[-1]) + 1
#             for next_char in VALID_VALUES[next_index:]:
#                 expanded.append(F"{pattern}{next_char}")
#         return expanded

#     def get_coord_map(arr, length):
#         # create a dictionary of all coordinates that possess each value in VALID_VALUES
#         mydict = {F"{x}": set() for x in VALID_VALUES}
#         numpy_iterator = np.nditer(arr, flags=['multi_index'], op_flags=['readwrite'])
#         for cell in numpy_iterator:
#             if len(str(cell)):
#                 for char in str(cell):
#                     mydict[char].add(numpy_iterator.multi_index[0])

#         # starting character
#         for idx in range(BOARD_SIZE):
#             pattern = VALID_VALUES[idx]
#             left_value = idx
#             for _ in range(length-1):
#                 left_value += 1
#                 for idy in range(left_value,BOARD_SIZE):


        

#     # initialize dictionary where the keys are all possible patterns
#     for row in range(BOARD_SIZE):
#         # char_to_coords = {F"{x}": set() for x in VALID_VALUES}
#         # numpy_iterator = np.nditer(arr[row,:], flags=['multi_index'], op_flags=['readwrite'])
#         # for cell in numpy_iterator:
#         #     if len(str(cell)):
#         #         for char in str(cell):
#         #             char_to_coords[char].add((row,numpy_iterator.multi_index[0]))

#         for idx in range(len(VALID_VALUES)):
#             for idy in range(idx+1, len(VALID_VALUES)):
#                 char1 = VALID_VALUES[idx]
#                 char2 = VALID_VALUES[idy]
#                 if len(char_to_coords[char1])==2 and char_to_coords[char1] == char_to_coords[char2]:
#                     print(F"{char1}{char2}: {char_to_coords[char1]}")


# hidden_naked_groups(possible)

---

## Load the puzzle

Before the puzzle is imported, two 2D numpy arrays are created. Both are BOARD_SIZE x BOARD_SIZE (traditional Sudoku is 9x9). The arrays are:

1. An array representing all "solved" cells is blank.
2. An array representing the possible values for each cell. Before the puzzle is imported, every value is possible for every cell, so the array looks something like the table below (only the first sector is shown).

| |  |  |  |
|:------:|:------:|:------:|:------:|
|  | 123456789 | 123456789 | 123456789 |
|  | 123456789 | 123456789 | 123456789 |
|  | 123456789 | 123456789 | 123456789 |

When the puzzle is read in, the "possible" cell is overwritten by the value specified in the puzzle. Most cells will remain set to every value as shown below. In the example below, the puzzle specifies only one value for the first sector while the rest of the cells are blank (so every value is possible).

| |  |  |  |
|:------:|:------:|:------:|:------:|
|  | 123456789 | 123456789 | 5 |
|  | 123456789 | 123456789 | 123456789 |
|  | 123456789 | 123456789 | 123456789 |



In [ ]:
possible = np.full((9,9),'123456789')
board = np.full((9,9),'')

# hard
puzzle = '2.......6.5..8..1...4...9...7.3.1......82.......7.5.3...9...4...8..1..5.6.......2'
# easy
puzzle = '5..98.67.6......31.2.613.4..968.21.7..8..5.9.7.319....962.7..1.1.5...76..7.5..9..'
# medium
puzzle = '.29.71..3..8...6..3...5....5.....97......4...4.75.8..1.6.42.3..2..9....6.916...52'
# hard
# puzzle = '..791.5....1.....3..9.4...2.4...83.....3.1....6..5...8.2..9...5...........4.8..7.'
# hardest ever
# puzzle = '1....7.9..3..2...8..96..5....53..9...1..8...26....4...3......1..4......7..7...3..'
# hard via sudoku.com
# puzzle = '..5........24.1.7.3.4...56...............79848...9..1....2..1...9..7...2.183...4.'
# expert via sudoku.com
# puzzle = '.....2..3.4...16.7..1.....4.8.1.....43.....62...7..............6.98.73.....3.47..'
# hard5
puzzle = '82.5.........3.257...67.9..4.61...3...........5..8419.9.2.1......57...2.......561'


for idx, value in enumerate(puzzle):
    if value in VALID_VALUES:
        row = idx // BOARD_SIZE
        col = idx - row * BOARD_SIZE
        possible[row,col] = value

solved = len(np.argwhere(get_length(possible)==1))
print(F"Puzzle starts with {solved} solved locations.")

print_one_board(possible)

---

## Solve the puzzle

### Step 1: Handle singletons

Look at every cell that has only one **possible** value. This is the only step that makes modifications to the actual board. Therefore, this will be the first and last steps of solving any puzzle.

This cell calls a function that runs repeatedly until it cannot simplify the board any more.

In [ ]:
print_two_boards(board,possible)
updates = handle_singletons(board,possible)
print(F"Updated {updates} singletons.")
if updates:
    print_two_boards(board,possible)

### Step 2: Handle row and column owners

In [ ]:
print_two_boards(board,possible)
updates = handle_rowcol_owners(possible)
print(F"Updated {updates} cells based on row and/or column owners.")
if updates:
    _ = handle_singletons(board, possible)
    print_two_boards(board,possible)

### Step 3: Handle naked groups

In [ ]:
print_two_boards(board,possible)
updates = handle_naked_groups(possible)
print(F"Updated {updates} cells based on naked groups.")
if updates:
    print_two_boards(board,possible)

### Step 4: Handle disguised singletons

In [ ]:
print_two_boards(board,possible)
disguised = handle_disguised_singletons(board,possible)
print(F"Located {disguised} disguised singletons.")
if disguised:
    print_two_boards(board,possible)